# TruthfulQA 

TruthfulQA เป็นการประเมินความถูกต้องของโมเดลภาษาในการตอบคำถามอย่างตรงไปตรงมา โดยมีคำถามทั้งหมด 817 ข้อ ครอบคลุม 38 หัวข้อ เช่น สุขภาพ กฎหมาย การเงิน และการเมือง

### Arguments
There are two optional arguments when using the TruthfulQA benchmark:

- [Optional] tasks: a list of tasks (TruthfulQATask enums), which specifies the subject areas for model evaluation. By default, this is set to all tasks. The complete list of TruthfulQATask enums can be found here.
- [Optional] mode: a TruthfulQAMode enum that selects the evaluation mode. This is set to TruthfulQAMode.MC1 by default. deepeval currently supports 2 modes: MC1 and MC2.

In [ ]:
from langchain_openai import AzureChatOpenAI
from deepeval.models.base_model import DeepEvalBaseLLM
import sys
sys.path.append('/opt/project/src/evaluate_llm/')
from api_key_config import settings
import os

os.environ["OPENAI_API_VERSION"] = settings.OPENAI_API_VERSION
os.environ["OPENAI_API_KEY"] = settings.OPENAI_API_KEY
os.environ["AZURE_OPENAI_ENDPOINT"] = settings.AZURE_OPENAI_ENDPOINT

class AzureOpenAI(DeepEvalBaseLLM):
    def __init__(
        self,
        model
    ):
        self.model = model

    def load_model(self):
        return self.model

    def generate(self, prompt: str) -> str:
        chat_model = self.load_model()
        return chat_model.invoke(prompt).content

    async def a_generate(self, prompt: str) -> str:
        chat_model = self.load_model()
        res = await chat_model.ainvoke(prompt)
        return res.content

    def get_model_name(self):
        return "Custom Azure OpenAI Model"

# Replace these with real values
custom_model = AzureChatOpenAI(
    deployment_name="gpt-35-turbo",
)
azure_openai = AzureOpenAI(model=custom_model)

In [ ]:
from deepeval.benchmarks import TruthfulQA
from deepeval.benchmarks.tasks import TruthfulQATask
from deepeval.benchmarks.modes import TruthfulQAMode

# Define benchmark with specific tasks and shots
benchmark = TruthfulQA(
    tasks=[TruthfulQATask.ADVERTISING, TruthfulQATask.FICTION],
    mode=TruthfulQAMode.MC2
)

# Replace 'mistral_7b' with your own custom model
benchmark.evaluate(model=mistral_7b)
print(benchmark.overall_score)